In [ ]:
import pandas as pd

#scores = pd.read_excel('analysis.xlsx')
scores = pd.read_csv('analysis_final2.csv')
scores

In [ ]:
n = 'complex'
#n = 'interface'

import pandas as pd
import plotly.graph_objects as go

def get_variant_order(df:pd.DataFrame) -> list:
    return sorted(df.columns.values, key=lambda v: int(v[1:-1]))

def get_annotation_for_heatmap(df:pd.DataFrame) -> list:
    d = df.to_numpy()
    annotations = []
    for i in range(d.shape[0]):
        for j in range(d.shape[1]):
            if abs(d[i,j]) > 2:
                text = f"<b>{d[i,j]:.1f}</b>"
            else:
                text = f"{d[i,j]:.1f}"
            annotations.append(dict(
                                    x=j,
                                    y=i,
                                    xref="x",
                                    yref="y",
                                    text=text,
                                    showarrow=False
                                    ))
    return annotations

ddG = scores.pivot_table(index=['model'], columns=['mutation'], values=f'{n}_ddG').round(1)
ddG = ddG[get_variant_order(ddG)]

fig = go.Figure(data=go.Heatmap(
    z=ddG,
    x=ddG.columns,
    y=ddG.index,
    hoverongaps=False,
    colorscale='temps'))
fig.data[0].update(zmin=-20, zmax=20)
fig.update_layout(title=f'ΔΔG relative to model used for variants',
                  annotations=get_annotation_for_heatmap(ddG),
                  xaxis = {'title': 'Variant'},
                  yaxis = {'title': 'Model'})
fig.write_image(f'heatmap_{n}.png', scale=3)
fig.show()